In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle

# Prediction 

In [2]:
from tensorflow.keras.models import load_model

ann_model=load_model('my_model.h5')

with open('label_encoder.pkl','rb') as file:
    gen_lbl_enc=pickle.load(file)

with open('ohe.pkl','rb') as file:
    country_one_hot_enc=pickle.load(file)

with open('scale.pkl','rb') as file:
    scale=pickle.load(file)

In [3]:
input_data={
    "credit_score":600,
    "country":"France",
    "gender":"Male",
    "age":40,
    "tenure":3,
    "balance":60000,
    "products_number":2,
    "credit_card":1,
    "active_member":1,
    "estimated_salary":50000
}

In [4]:
df_test=pd.DataFrame([input_data])
df_test

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary
0,600,France,Male,40,3,60000,2,1,1,50000


In [5]:
df_test['gender']=gen_lbl_enc.fit_transform(df_test['gender'])
df_test

,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary
0,600,France,0,40,3,60000,2,1,1,50000


In [6]:
country_ohe=country_one_hot_enc.transform([df_test['country']])
country_ohe.toarray()

c:\Users\ADITYA SWARUP\New folder\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([[1., 0., 0.]])

In [7]:
country_df=pd.DataFrame(country_ohe.toarray(),columns=country_one_hot_enc.get_feature_names_out(['country']))
country_df

,country_France,country_Germany,country_Spain
0,1.0,0.0,0.0


In [8]:
df_test=pd.concat([df_test.drop('country',axis=1),country_df],axis=1)
df_test

,credit_score,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,country_France,country_Germany,country_Spain
0,600,0,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [9]:
df_scaled=scale.fit_transform(df_test)
df_scaled

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [10]:
prediction=ann_model.predict(df_scaled)
prediction

1/1 [==============================] - 0s 215ms/step


array([[0.23198374]], dtype=float32)

In [11]:
prediction_prob=prediction[0][0]
prediction_prob

0.23198374

In [12]:
if prediction_prob>0.6:
    print("Customer is likely to churn")
else:
    print('Customer is unlikely to churn')

Customer is unlikely to churn
